In [1]:
import findspark
findspark.init('/Users/kiranrudresha/Documents/spark/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession



In [2]:
spark = SparkSession.builder.appName('nlp3').getOrCreate()

In [5]:
sms_data = spark.read.csv('/Users/kiranrudresha/Documents/spark/pyspark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning'
               '/Natural_Language_Processing/smsspamcollection/SMSspamcollection', inferSchema=True,
                sep = '\t')

In [6]:
sms_data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [9]:
data = sms_data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [15]:
data = data.withColumn('length', length(data['text']))

In [14]:
from pyspark.sql.functions import length

In [16]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [17]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [18]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer,
                                IDF, StringIndexer)

In [26]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremover = StopWordsRemover(inputCol='token_text', outputCol='stop_text')
count_vec = CountVectorizer(inputCol='stop_text', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
class_idx = StringIndexer(inputCol='class', outputCol='label')

In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
vec_asm = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [22]:
from pyspark.ml.classification import NaiveBayes

In [24]:
from pyspark.ml import Pipeline

In [27]:
pipe_data = Pipeline(stages = [class_idx,tokenizer,stopremover, count_vec, idf, vec_asm])

In [28]:
pipe_model = pipe_data.fit(data)

In [29]:
clean_data = pipe_model.transform(data)

In [31]:
clean_data.head(1)

[Row(class='ham', text='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', length=111, label=0.0, token_text=['go', 'until', 'jurong', 'point,', 'crazy..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'cine', 'there', 'got', 'amore', 'wat...'], stop_text=['go', 'jurong', 'point,', 'crazy..', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'cine', 'got', 'amore', 'wat...'], c_vec=SparseVector(13423, {7: 1.0, 11: 1.0, 31: 1.0, 61: 1.0, 72: 1.0, 344: 1.0, 625: 1.0, 731: 1.0, 1409: 1.0, 1598: 1.0, 4485: 1.0, 6440: 1.0, 8092: 1.0, 8838: 1.0, 11344: 1.0, 12979: 1.0}), tf_idf=SparseVector(13423, {7: 3.1126, 11: 3.2055, 31: 3.822, 61: 4.2072, 72: 4.322, 344: 5.4072, 625: 5.918, 731: 6.1411, 1409: 6.6801, 1598: 6.8343, 4485: 7.5274, 6440: 7.9329, 8092: 7.9329, 8838: 7.9329, 11344: 7.9329, 12979: 7.9329}), features=SparseVector(13424, {7: 3.1126, 11: 3.2055, 31: 3.822, 61: 4.

In [32]:
data = clean_data.select('label', 'features')

In [33]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [34]:
train, test = data.randomSplit([0.7,0.3])

In [35]:
train.count()

3914

In [36]:
test.count()

1660

In [37]:
nb = NaiveBayes()
# default values best classification model for NLP 

In [39]:
spam_detect = nb.fit(train)

In [40]:
spam_final = spam_detect.transform(test)

In [41]:
spam_final.show()


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,14,31...|[-215.77787925195...|[1.0,9.8643541109...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-686.82563905027...|[1.0,3.3988875691...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-689.65053269272...|[1.0,1.8356417318...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1006.2405802402...|[1.0,6.6865750175...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1542.6711469576...|[0.15952789849449...|       1.0|
|  0.0|(13424,[0,1,31,43...|[-344.09276199250...|[1.0,3.3851240426...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-615.64719562686...|[0.99999220628563...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-267.20831345308...|[4.00935218056017...|       1.0|
|  0.0|(13424,[0,2,3,6,9...|[-3293.9834624523...|[1.0,1.4623688793...|       0.0|
|  0.0|(13424,[0

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 

In [43]:
evaluator = MulticlassClassificationEvaluator()

In [47]:
acc = evaluator.evaluate(spam_final)

In [48]:
aoc

0.9197478510533361